In [21]:
import requests
import pandas as pd
import spacy
from collections import Counter
import re




In [22]:
url = "https://newsapi.org/v2/everything"
params = {
    'q': 'olympics',
    'from': '2024-08-03',
    'sortBy': 'popularity',
    'apiKey': 'da7394bf6c484381bbf90c62c919273e',
    'sources' : 'espn,associated-press'
}
response = requests.get(url, params=params)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Convert the JSON response to a DataFrame
    df = pd.json_normalize(data['articles'])
    
    df = df[['author', 'publishedAt', 'source.name', 'title', 'url', 'content']]
    
    # Rename the columns to desired names
    df.rename(columns={'publishedAt': 'date', 'source.name': 'news source', 'url': 'link'}, inplace=True)
    
    # Print the DataFrame
    display(df[df['news source'] == 'Associated Press'])

else:
    print(f"Failed to retrieve data: {response.status_code}")
    
    

,author,date,news source,title,link,content
35,GREG BEACHAM,2024-08-03T14:13:38Z,Associated Press,Banned governing body that's fueling outcry ha...,https://apnews.com/article/olympics-2024-kheli...,"VILLEPINTE, France (AP) Nearly 17 months ago i..."
46,AP,2024-08-06T18:03:00Z,Associated Press,"Meet Mourad Merzouki, Choreographer Of The Oly...",https://apnews.com/article/olympics-2024-moura...,"CRETEIL, France (AP) In a sweltering enclosed ..."
66,ap news,2024-08-03T17:00:03Z,Associated Press,Algerian boxer clinches medal after gender out...,https://apnews.com/article/olympics-2024-imane...,"VILLEPINTE, France (AP) Boxer Imane Khelif of ..."


In [23]:
nlp = spacy.load('en_core_web_sm')
word_counter = Counter()

def process_text(text):
    text = re.sub(r'\r\n|\r|\n', ' ', text).lower()
    doc = nlp(text.lower())  # Convert text to lowercase and process with spaCy
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]  # get rid of stopwords
    # TODO set names as one word
    return tokens


for content in df['content'].dropna().head():
    tokens = process_text(content)
    word_counter.update(tokens)


In [24]:
word_counter.most_common(10)

[('chars', 5),
 ('olympics', 4),
 ('2024', 3),
 ('august', 3),
 ('tennis', 3),
 ('djokovic', 3),
 ('men', 3),
 ('halfway', 2),
 ('bring', 2),
 ('exciting', 2)]

In [25]:
import transformers
import torch

## Here you paste your cloned repos location
model_id = "/Users/gabeschwartz/Documents/GitHub/NewsProject/Meta-Llama-3.1-8B-Instruct" 

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [27]:
import datetime as dt

dt.today()

AttributeError: module 'datetime' has no attribute 'today'